In [82]:
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome import options
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import Select
from bs4 import BeautifulSoup
from time import sleep
import pandas as pd
import html5lib
import openpyxl
import datetime 
from datetime import date
import pyodbc
import sqlalchemy as db
from sqlalchemy.orm import sessionmaker

In [69]:
options = Options()
options.headless = False
navegador = webdriver.Chrome(options=options)
link='http://sifserver.gonsys.com:15001/SIF/authentication/login'
navegador.get(url=link)
sleep(3)
inputUsuario = navegador.find_element(by=By.ID,value='txtUser')
inputUsuario.send_keys('')
inputSenha = navegador.find_element(by=By.ID,value='txtPassword')
inputSenha.send_keys('')

sleep(2)
botao = navegador.find_element(by=By.NAME,value='sign-in')
botao.click()
sleep(2)

rps = navegador.find_element(by=By.LINK_TEXT,value= 'RPS')
rps.click()
sleep(5)

dropdow = navegador.find_element(By.ID,'ddlEmpresa')
dropdow = Select(dropdow)
dropdow.select_by_value('1')
sleep(5)

pesquisar = navegador.find_element(by=By.XPATH,value='/html/body/app-root/div[2]/app-rps/body/form/div[2]/button')
pesquisar.click()

sleep(5)
resultado = navegador.find_element(by=By.XPATH,value='/html/body/app-root/div[2]/app-rps/body/div[2]/div/div/table')
html = resultado.get_attribute("outerHTML")
sleep(5)
soup = BeautifulSoup(html,'html.parser')
relatorio = soup.find(name='table')
dataframe = pd.read_html(str(relatorio))[0]
dataframe = dataframe

data_atual = date.today()
data_atual = data_atual.strftime('%d/%m/%Y')

hora_atual = datetime.datetime.now()
hora_atual = hora_atual.strftime('%H:%M')

controle = dataframe.assign(Data = data_atual,Hora = hora_atual,Usuário= ['Guilherme'])

controle = controle.drop(columns=['Garagem','Empresa','Estado','Cidade'])

controle = controle[['Data','Hora','Usuário','Qtde RPS','Qtde Transmitido','Qtde Erro','Qtde Transmitir','Total Transmitido','Total com Erro','Total a Transmitir','Total Geral']]

controle = controle.rename(columns = {'Data':'Acompanhamento de Atualização SIF GRU','Hora':'Unnamed: 1','Usuário':'Unnamed: 2','Qtde RPS':'Unnamed: 3','Qtde Transmitido':'Unnamed: 4','Qtde Erro':'Unnamed: 5','Qtde Transmitir':'Unnamed: 6','Total Transmitido':'Unnamed: 7','Total com Erro':'Unnamed: 8','Total a Transmitir':'Unnamed: 9','Total Geral':'Unnamed: 10'})
#Conectando ao Banco
def DadosMensalistas(query):
    server = '127.0.0.1,1434' 
    database = 'SIF_Prod' 
    username = 'sqlserver' 
    password = '@dm1nSIF'  
    cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
    cursor = cnxn.cursor()
    query = query
    dataframe_banco = pd.read_sql(query, cnxn)
    return dataframe_banco

dataframe_banco = DadosMensalistas('select p.Descricao,max(DataEmissao) data from RPS r join Produto p on r.idproduto=p.IDProduto  group by r.IDProduto,p.Descricao')


# Método "T" transpondo linhas em colunas para adcionar ao dataframe do relatorio
colunas = dataframe_banco.T 
controle = colunas.assign('Rotativo','Mensalista PF','Mensalista PJ','Selo')

controle.to_excel('relatorio.xlsx' ,encoding='UTF=8',index=False)

pd.read_excel('Acompanhamento SIF GRU.xlsx')

def leitor_xlsx(df):
    df_strd = pd.read_excel('Acompanhamento SIF GRU.xlsx')
    df= df_strd.append(df)
    writer = pd.ExcelWriter('Acompanhamento SIF GRU.xlsx',engine='xlsxwriter')
    df.to_excel(writer,sheet_name='Plan1',index=False)
    writer.save()

status_atual = pd.read_excel(r"relatorio.xlsx",sheet_name = 'Sheet1',)


leitor_xlsx(status_atual)

C:\Users\guilherme\AppData\Local\Temp\ipykernel_21696\308558210.py:2: DeprecationWarning: headless property is deprecated, instead use add_argument('--headless') or add_argument('--headless=new')
  options.headless = False
c:\Users\guilherme\OneDrive\Área de Trabalho\GUILHERME\PROJETOS PYTHON\Auto_Relatorio\.venv\lib\site-packages\pandas\util\_decorators.py:211: FutureWarning: the 'encoding' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'encoding'
  return func(*args, **kwargs)
C:\Users\guilherme\AppData\Local\Temp\ipykernel_21696\308558210.py:58: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df= df_strd.append(df)
C:\Users\guilherme\AppData\Local\Temp\ipykernel_21696\308558210.py:61: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()


In [117]:
colunas = dataframe_banco.T 
colunas  = colunas.drop(columns=['Descricao'])
controle = controle.assign(colunas)


KeyError: "['Descricao'] not found in axis"

In [151]:
teste = dataframe_banco.drop(columns='Descricao')

In [155]:
informe = teste.T

formatado = dataframe.assign(informe)

TypeError: DataFrame.assign() takes 1 positional argument but 2 were given